In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info = True, as_supervised = True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [3]:

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale (image,label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [4]:
BUFFER_SIZE = 10000
shuffle_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)
validation_data = shuffle_train_and_validation_data.take(num_validation_samples)
train_data = shuffle_train_and_validation_data.skip(num_test_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_target = next(iter(validation_data))

In [5]:
input_size = 784
output_size = 10
hidden_layer_size = 250

model = tf.keras.Sequential([ 
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),    
                            tf.keras.layers.Dense(hidden_layer_size, activation='tanh'),  
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

In [6]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
num_epochs = 100
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
model.fit(train_data, epochs = num_epochs, callbacks=(early_stopping), validation_data = (validation_inputs, validation_target), verbose = 2)

Epoch 1/100
500/500 - 5s - loss: 0.2431 - accuracy: 0.9259 - val_loss: 0.1071 - val_accuracy: 0.9682 - 5s/epoch - 10ms/step
Epoch 2/100
500/500 - 4s - loss: 0.0951 - accuracy: 0.9698 - val_loss: 0.0977 - val_accuracy: 0.9708 - 4s/epoch - 9ms/step
Epoch 3/100


In [ ]:
# r1(100-2 layers) = 97,48%; r2(200-2 layers)= 98,23%; r3(100-3 layers);
# r4(200-5 layers) = 98,02%; r4(sigmoid)=95,85%; r5(relu+tanh)=97,85%
# r6(batchsize=10000) = 84,53%; r7(lR=0.0001)=94,47%; r8(LR=0,02)=95,98%

## hidden layer size = 250, 2 layers relu, 1 layer tanh

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)

In [ ]:
print('Test loss: {0:.2f}. Test accuracy: {01:.2f}%'.format(test_loss,test_accuracy*100.))